### Описание проекта

**Тема:** Исследование уровня NPS для телекоммуникационной компании

**Описание:**

NPS - это индекс потребительской лояльности. Он показывает, насколько клиенты довольны услугами компании. Чем индекс выше, тем больше повторных покупок и обращений по рекомендациям.

Чтобы определить уровень лояльности, клиентам задают классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым». 

В зависимости от оценки респонденты распределяются по трём категориям:

    9-10 баллов — «cторонники» (англ. promoters);
    7-8 баллов — «нейтралы» (англ. passives);
    0-6 баллов — «критики» (англ. detractors).
    
Итоговое значение NPS рассчитывается по формуле: % «сторонников» - % «критиков».

**Данные:** 

    * таблица user - содержит основную информацию о клиентах
    * таблица location - справочник территорий, в которых телеком-компания оказывает услуги
    * таблица age_segment - данные о возрастных сегментах клиентов
    * таблица traffic_segment - данные о выделяемых сегментах по объему потребляемого трафика
    * таблица lifetime_segment - данные о выделяемых сегментах по количеству месяцев "жизни" клиента
   
**Ход работы:**

Создам csv-файл со следующей информацией:

    - идетификатор клеинта
    - количество дней "жизни" клиента
    - новый/старый клиент
    - возраст
    - пол
    - тип операционной системы
    - тип устройства
    - страна проживания
    - город проживания
    - возрастной сегмент
    - сегмент по объему потребляемого трафика
    - сегмент по количеству месяцев "жизни"
    - оценка клиента в NPS-опросе
    - группа клиента (сторонники, нейтралы, критики)

В программе Tableau создам дашборд с информацией о текущем уровне NPS среди клиентов в зависимости от пользовательских признаков.

Подготовлю презентацию с выводами.

Опишу типичного клиента среди сторонников.

In [1]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [3]:
query = """
SELECT u.user_id, 
       u.lt_day,
       CASE
           WHEN u.lt_day <= 365 THEN 'new'
           ELSE 'old'
       END AS is_new,
       u.age,
       CASE 
           WHEN u.gender_segment = 1 THEN 'женщина'
           WHEN u.gender_segment = 0 THEN 'мужчина'
       END AS gender_segment,
       u.os_name,
       u.cpe_type_name,
       l.country,
       l.city,
       SUBSTRING(a_s.title,3) AS age_segment,
       SUBSTRING(t_s.title,3) AS traffic_segment,
       SUBSTRING(l_s.title,3) AS lifetime_segment,
       u.nps_score,
       CASE
           WHEN u.nps_score BETWEEN 9 AND 10 THEN 'Сторонники'
           WHEN u.nps_score BETWEEN 7 AND 8 THEN 'Нейтралы'
           WHEN u.nps_score BETWEEN 0 AND 6 THEN 'Критики'
       END AS nps_group
FROM user AS u
INNER JOIN location AS l ON u.location_id=l.location_id 
INNER JOIN age_segment AS a_s ON u.age_gr_id=a_s.age_gr_id
INNER JOIN traffic_segment AS t_s ON u.tr_gr_id=t_s.tr_gr_id
INNER JOIN lifetime_segment AS l_s ON u.lt_gr_id=l_s.lt_gr_id
"""

In [4]:
df = pd.read_sql(query, engine)
df.head(10)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,old,45.0,женщина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,Сторонники
1,A001WF,2344,old,53.0,мужчина,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,Сторонники
2,A003Q7,467,old,57.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,Сторонники
3,A004TB,4190,old,44.0,женщина,IOS,SMARTPHONE,Россия,РостовнаДону,35-44,0.1-1,36+,10,Сторонники
4,A004XT,1163,old,24.0,мужчина,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,Сторонники
5,A005O0,5501,old,42.0,женщина,ANDROID,SMARTPHONE,Россия,Омск,35-44,5-10,36+,6,Критики
6,A0061R,1236,old,45.0,мужчина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,10-15,36+,10,Сторонники
7,A009KS,313,new,35.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,35-44,45-50,7-12,10,Сторонники
8,A00AES,3238,old,36.0,женщина,ANDROID,SMARTPHONE,Россия,СанктПетербург,35-44,1-5,36+,10,Сторонники
9,A00F70,4479,old,54.0,женщина,ANDROID,SMARTPHONE,Россия,Волгоград,45-54,15-20,36+,9,Сторонники


In [ ]:
df.to_csv('telecomm_csi_tableau.csv', index=False) # создание файла с датафреймом

### ссылка на дашборд
Ниже ссылка на дашборд "Текущий уровень NPS", ссылка на презентацию: "Исследование уровня NPS"

ССЫЛКА на дашборд: https://public.tableau.com/views/Project2_dashboard_04_07/NPS_6?:language=en-US&publish=yes&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link

ОТДЕЛЬНАЯ ССЫЛКА на презентацию: https://public.tableau.com/views/Project2_story_04_07/NPS_7?:language=en-US&publish=yes&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link

### Общий вывод по исследованию:

* общий уровень NPS равен 21,9%


* всего опрошено чуть более 500 тысяч человек
* к группе сторонников относится 52,5% респондентов
* к группе критиков - 30,7% респондентов
* "старых" пользователей подавляющее большинство - 83%, а "новых" - всего 17%
* больше всего опрошеннных в Москве - 66160 человек. Более чем в два раза меньше опрошено в Санкт-Петербурге, Новосибирске, Екатеринбурге и Казани
* женщин опрашивали чаще, чем мужчин, особенно это заметно в категории 45+.


* Уровень NPS выше среди женщин (25,55%), чем среди мужчин (17.7%)
* Среди женщин число сторонников - 55%, а критиков  - 30%
* Среди мужчин число сторонников - 50%, а критиков - 32%
* Среди сторонников женщин - 57%, а мужчин - 43%
* Среди критиков женщин - 52%, а мужчин - 48%


* Уровень NPS имеет отрицательное значение только в возрастной категории "до 16 лет", т.е. тут уровень критиков выше, чем сторонников
* Во всех остальных возрастных категориях наблюдается рост уровня NPS с увеличением возраста респондентов
* Наибольшее число сторонников в возрастной категории "66+" - 70% среди всех респондентов этого возраста
* Наибольшее число критиков в возрастной категории "до 16 лет" - почти 41,2% критиков среди всех респондентов этого возраста
* Среди всех опрошенных наибольшее число сторонников в возрастной категории "35-44" - 17,41%, наибольшее число критиков в категории "25-34" - почти 11%
* Почти 36% всех критиков из возрастной категории "25-34"
* 33% сторонников относятся к категории "35-44" 


* Уровень NPS выше среди новых клиентов (35,6% против 19%)
* Среди сторонников старых клиентов 80,37%, новых клиентов - 19,63%
* Среди критиков старых клиентов 86,25%, новых клиентов - 13,75%
* Среди старых клиентов число сторонников почти 51%, а критиков - почти 32%
* Среди новых клиентов число сторонников 60,22%, а критиков - 24,6%
* Уровень NPS самый высокий среди клиентов, которые пользуются услугами компании всего 1 месяц - 56,5%. С каждым последующим месяцем пользования уровень NPS  неуклонно падает, достигая значения 15,75% среди тех, кто пользуется услугами более 36 месяцев.


* Самый высокий уровень NPS в Москве.
* В каждом конкретном городе число сторонников в районе 50%, а число критиков в районе 30%
* Наибольшее число сторонников в Москве (56%) из всех опрошенных москвичей
* Наибольшее число критиков в Челябинске (35%) из всех опрошенных челябинцев


На основе проведенного опроса и представленных данных можно составить портрет типичного клиента, относящегося к группе сторонников компании:
* это женщина
* в возрасте от 35 лет
* пользуется услугами первый месяц (но не более 1 года)
* живет в Москве (или возможно в Ростове-на-Дону, Казани, Омске, Самаре или Санкт-Петербурге)